# Capstone Project (Mx Data)
## 1. Setup and Reading Files

In [15]:
import pandas as pd
import datetime as dt
import numpy as np
import calendar

In [43]:
df = pd.read_csv('mx_raw.csv')
pd.set_option('display.max_rows', df.shape[0]+1)
df.head()

,Asset ID,Asset Description,Work ID,Status,Event ID,Event Name,Actual Duration (HRS),date,mx type,mx time,Priority Number,Event Priority
0,AHU1,AHU,MNT-213824,Closed,ME-007243,Monthly Maintenance,0.5,12/28/2020,ME,Monthly,1,3.0
1,AHU1,AHU,MNT-213615,Closed,ME-007916,Quarterly Maintenance,0.5,12/18/2020,ME,Quarterly,1,3.0
2,AHU1,AHU,MNT-210761,Closed,ME-007917,Semi-Annual Maintenance,1.0,9/23/2020,ME,Semi-Annual,1,2.0
3,AHU1,AHU,MNT-201361,Closed,ME-008544,Annual Maintenance,1.0,11/20/2019,ME,Annual,1,2.0
4,AHU1E,AHU,MNT-200719,Closed,ME-007244,Monthly Maintenance,1.0,10/30/2019,ME,Monthly,1,3.0


## 2. Find Upcoming Events for Subsequent Month

In [44]:
# convert date column to datetime type
df.date = pd.to_datetime(df.date)

In [45]:
# find unique mx times
df['mx time'].unique()

array(['Monthly', 'Quarterly', 'Semi-Annual', 'Annual', '2yr', 'Weekly',
       'Daily', '2 Week', 'Bi-Weekly'], dtype=object)

In [46]:
# create the new date column by add the date and mx time columns
# convert mx time to hours
# create a list of our conditions
conditions = [
    (df['mx time'] == '2yr'),
    (df['mx time'] == 'Annual'),
    (df['mx time'] == 'Semi-Annual'),
    (df['mx time'] == 'Quarterly'),
    (df['mx time'] == 'Monthly'),
    (df['mx time'] == '2 Week'),
    (df['mx time'] == 'Weekly'),
    (df['mx time'] == 'Bi-Weekly'),
    (df['mx time'] == 'Daily')
    ]

# create a list of the values we want to assign for each condition
values = [17520,
          8760,
          4380,
          2190,
          730,
          336,
          168,
          84,
          24
         ]
# create new date column
df['mx_due_date'] = df.date + pd.to_timedelta(np.select(conditions, values), unit='h')

In [47]:
# find today 
today = dt.date.today()

# slice all mx due next month or past due
next_month_mx = df.loc[(df.mx_due_date.dt.month == today.month + 1) | (df.mx_due_date < pd.to_datetime(today))]

# find sum of avg duration
tot_mx_hrs = sum(next_month_mx['Actual Duration (HRS)'])

## 3. Calculate total MX hours available

In [48]:
# pd.to_datetime(today).weekday()
next_month = today.month + 1
year = today.year

weekdays = 0
weekends = 0
cal = calendar.Calendar()

# iterate through weeks in month
for week in cal.monthdayscalendar(year, next_month):
    for i, day in enumerate(week):
        # check if in month and weekend
        if day != 0 and i >= 5:
            weekends += 1
        # check if in month and weekday
        elif day!= 0 and i < 5:  
            weekdays += 1

tot_man_hrs = weekdays * 32 + weekends * 18

## 4. Can the MX Crew Finish All Events?
#### 3.1 Yes: Write Events to a CSV
#### 3.2 No: Pick Events by Priority and Write Priority Events to a CSV

In [50]:
# check if MX team can complete all work
if tot_mx_hrs <= tot_man_hrs:
    # display next month mx
    display(next_month_mx.loc[:,['Asset ID', 'mx_due_date', 'Priority Number']].sort_values('Priority Number'))
else:  # cannot complete all required work
    print('Need to prioritize')

Need to prioritize
